# from txt file from national data buoy center

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np

In [ ]:
df = pd.read_csv("current_netcdfs/ndbc_46025_wind.txt", r" +")
# units row, drop
df = df.drop(0)
df = df.drop(df[df["WDIR"] == "MM"].index)
df

In [ ]:
# notes: wind direction is where wind is coming from, pointing in opposite direction
# 3% of wspd added to ocean vector
uw_obs = -df['WSPD'].astype(float).values * np.sin(np.deg2rad(df['WDIR'].astype(float).values))
vw_obs = -df['WSPD'].astype(float).values * np.cos(np.deg2rad(df['WDIR'].astype(float).values))

In [ ]:
# the column names are different oh god why
timestamp = df["%YY"] + "-" + df["0.0"] + "-" + df["DD"] + "T" + df["hh"] + ":" + df["mm"]
timestamp = pd.to_datetime(timestamp)

In [ ]:
new_df = pd.DataFrame({"time": timestamp, "u": uw_obs, "v": vw_obs})
new_df = new_df.sort_values(by="time")
new_df

In [ ]:
u = xr.DataArray(new_df["u"], coords={"time": new_df["time"]})
v = xr.DataArray(new_df["v"], coords={"time": new_df["time"]})

In [ ]:
ds = xr.Dataset({"U": u, "V": v})
ds

In [ ]:
ds.to_netcdf("current_netcdfs/ndbc_46025_wind.nc")

# different format, .mat file lol

please help

In [1]:
from scipy.io import loadmat
import numpy as np
import xarray as xr
import datetime

winds = loadmat("current_netcdfs/GFS_PointSource_winds_188_3325.mat")

In [2]:
converted = [datetime.datetime.strptime(t, '%m-%d-%Y %H:%M') for t in winds["time_str"]]
times = np.array(converted, dtype=np.datetime64)

In [3]:
u = xr.DataArray(winds["u"].squeeze(), coords={"time": times})
v = xr.DataArray(winds["v"].squeeze(), coords={"time": times})

In [4]:
ds = xr.Dataset({"U": u, "V": v})
ds

<xarray.Dataset>
Dimensions:  (time: 132)
Coordinates:
  * time     (time) datetime64[ns] 2021-10-01 ... 2021-10-06T11:00:00
Data variables:
    U        (time) float64 5.678 5.148 4.966 4.344 ... 1.143 1.093 1.002 0.4962
    V        (time) float64 -0.7658 -2.407 -2.82 ... 0.05237 0.02239 0.1385

In [5]:
ds.to_netcdf("current_netcdfs/GFS_PointSource_Winds_188_3325.nc")